## Read everything in the problem statement

If you need help, try the prepromt on [comfitlib.com](https://comfitlib.com/) and paste it into your LLM of choice (e.g., GPT UiO).

![](../preprompt_demo.gif)

**If you are running Google Colab, uncomment the following code to install the required libraries**

In [ ]:
# !pip install comfit==1.9.2 -q # Uncomment and run this cell if you are using Google Colab

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import comfit as cf

# Module 4

Goals for the module:
* Use the Born rule to compute the probability of observing a particle.
* Demonstrate examples of quantum tunnelling.

## Problem 1

In this problem, you will use the Born rule to compute the probability of measuring a particle in a region.

#### Subproblem a)
* State the Born rule. 
* For a particle in one dimension - what is the probability of measuring the particle to lie somewhere in an interval $[x_1, x_2]$?

#### Subproblem b)

In this problem, you will consider a wavepacket that is initialized at $x=0$ with initial velocity $1$.
<!-- * Write a function that takes in a wavefunction `psi` and positions `x1` and `x2`, and computes the probability of observing the particle between `x1` and `x2`. -->
* At time $t=0$, what is the probability that we observe the particle in the interval [-2, 2]? Feel free to use the function `integrate_interval` defined below!
* Suppose that the system is allowed to evolve undisturbed for 30 timesteps. What is the probability of measuring the particle in [-2, 2] now? Use `qm.evolve_schrodinger` to evolve the system.

Note: You can plot the complex wavefunction using `qm.plot_complex_field`, or the probability density (a real field) using `qm.plot_field`. We encourage you to do this:)

In [ ]:
# Initialize a 1-Dimensional system 
dt = 0.1
xRes = 201
xlim = [-10,10]
dx = (xlim[1]-xlim[0])/(xRes-1)

qm = cf.QuantumMechanics(1, xlim=xlim, dt=dt, xRes=xRes)
qm.conf_initial_condition_Gaussian(position=0, width=1, initial_velocity=1)
psi_initial = qm.psi

In [ ]:
def integrate_interval(field: np.ndarray, x_start: float, x_end: float, qm: cf.QuantumMechanics) -> float:
    """Integrates a field over a given interval

    Args:
        field (np.ndarray): a field to integrate
        x_start (float): start of the interval
        x_end (float): end of the interval
        qm (cf.QuantumMechanics): quantum mechanics object

    Returns:
        float: the integral of the field over the interval
    """
    x = qm.x
    dx = x[1] - x[0]
    is_in_interval = (x >= x_start) & (x <= x_end) # Boolean array with True where x is in the interval
    return np.sum(field[is_in_interval]) * dx

## Problem 2

You might have heard of the term "quantum tunnelling" in popular science. But what is quantum tunnelling really? 

Quantum tunnelling is a name for when a particle moves to a region that is classically forbidden (ie. the particle wouldn't be able to go there, according to Newton's laws of motion), in a way that is predicted by quantum mechanics.
In this problem, we will see an example of quantum tunnelling! We will consider the following setup:

A particle is located at $x=-10$, with mass $1$ and initial velocity $1$. There is a potential wall, given by
$$
V(x) = \begin{cases}
1, & \text{if } 0 \le x \le 5, \\
0, & \text{otherwise.}
\end{cases}
$$

Subproblem a)

* Describe how the particle will behave, if we treat it classically. How long does it take for it to hit the wall? What happens then? Use an energy conservation argument.

Subproblem b)

We will now treat this system quantum mechanically. We initialize a wavepacket representing the particle, with mass $1$ and initial velocity $1$. 

* Evolve the system for 100 timesteps (which correspons to $t=10$) using `qm.evolve_schrodinger`. Plot the wavefunction using `qm.plot_complex_field`.
* Evolve the system for another 100 timesteps and plot the wavefunction.
* What happens as the packet hits the wall? Compare with the classical case
* Compute the probability that the particle has passed through the potential wall after the full 200 timesteps of time evolution.


In [ ]:
# Initialize a 1-Dimensional system 
dt = 0.1
xRes = 1001
xlim = [-50,50]
dx = (xlim[1]-xlim[0])/(xRes-1)

qm = cf.QuantumMechanics(1, xlim=xlim, dt=dt, xRes=xRes)
qm.conf_initial_condition_Gaussian(position=-10, width=1, initial_velocity= 1)
V = 1 * (qm.x > 0 ) * (qm.x < 5)
qm.V_ext = V
fig, ax = qm.plot_field(V, size=(400, 400))
fig

In [ ]:
fig, ax = qm.plot_complex_field(qm.psi) # Plot the initial wavefunction
qm.plot_field(qm.V_ext, fig=fig, ax=ax)
fig

#### Subproblem c)

You will now run two different variations of the experiment setup in subproblem b).

* Make the obstacle twice as wide (i.e., from $x=-5$ to $x=+5$). What is the tunneling probability after 200 timesteps?
* Make the obstacle twice as "high", i.e. double `qm.V_ext`. What is the tunneling probability after 200 timesteps? (For this experiment, you should use same obstacle width as in subproblem b).

#### Subproblem d)

* Run the below code, which generates an animation of the time evolution of the system. What do you see?

In [ ]:
# Initialize a 1-Dimensional system 
dt = 0.1
xRes = 1001
xlim = [-50,50]
dx = (xlim[1]-xlim[0])/(xRes-1)

qm = cf.QuantumMechanics(1, xlim=xlim, dt=dt, xRes=xRes)
qm.conf_initial_condition_Gaussian(position=-10, width=1, initial_velocity= 1)
V = 1 * (qm.x > 0 ) * (qm.x < 5)
qm.V_ext = V
fig, ax = qm.plot_field(V, size=(400, 400))
fig

plot_height = np.max(np.abs(qm.psi))
for n in range(40):
    print(n, end='\r')
    qm.evolve_schrodinger(5)
    fig, ax = qm.plot_complex_field(qm.psi, size=(400, 400)) # Plot the initial wavefunction
    qm.plot_field(qm.V_ext*plot_height, fig=fig, ax=ax, vlim=[0,plot_height])
    qm.plot_save(fig, n)
cf.tool_make_animation_gif(n)
print("Done! Check you file system (folder icon on the left in Colab).")


## Problem 3


We are now living in a highly digital world. All the digital devices - computers, tablets, TV's, and so on - are powered by microchips. And zooming further in, these microchips consist of a large amount of transistors.

Moore's law is the observation that the number of transistors we can fit on a microchip approximately doubles every two years. Chip manufacturers have been able to achieve this by creating smaller and smaller transistors. 

Problem arises when the transistors become so small that electrons can quantum tunnel. This is because transistors rely on stopping and starting electrical signals, i.e., the flow of electrons. If the electrons start tunneling through the components of our computer, then Houston, we have a problem.

We can model an electron closed off by a transistor by placing it in a potential well. 
This is what we will consider in this problem.

In [ ]:
# Initialize a 1-Dimensional system 
dt = 0.1
xRes = 501
xlim = [-50,50]

qm = cf.QuantumMechanics(1, xlim=xlim, dt=dt, xRes=xRes)

transistor_width = 20

V = 1/2 * (
    0.5*np.tanh((qm.x + transistor_width/2)/2) 
    - 1.5*np.tanh((qm.x + 30)/2) 
    + 1.5*np.tanh((qm.x - 30)/2) 
    - 0.5*np.tanh((qm.x - transistor_width/2)/2) 
    ) + 1.5

qm.V_ext = V

fig, ax = qm.plot_field(V, size=(400, 400))
fig.update_layout(yaxis_title="Potential")
fig

#### Subproblem a)


First we will consider the classical behavior of an electron in this potential. Specifically, consider an electron with mass $m=1$, initial velocity $v_{init} = 1/2$, and initial position $x_{init} = -20$.

* According to classical mechanics, will the electron  the leftmost potential well?

#### Subproblem b)

Now we will consider the quantum mechanical version of this system. We initialize a wave packet centered at $x_{init}=-20$, with initial velocity $v_{init} = 0.5$. We plot the potential along with the initial wavefunction below. 

In [ ]:
qm.conf_initial_condition_Gaussian(position=-20, width=2.5, initial_velocity= 0.5)

fig, ax = qm.plot_field(V, size=(400, 400)) # Plot potential
qm.plot_complex_field(qm.psi, size=(400, 400), fig=fig, ax=ax) # Plot initial wavefunction
fig

<!-- Plot initial energy, dotted line -->

* Run the below code, which generates a gif of the system as it evolves in time.
* Modify the code so that you compute the probability of tunnelling to the interval [10, 30] as a function of time steps.
* Plot the tunnelling probability as a function of time steps evolved.
<!-- * Briefly comment what you see. Is this different to the classical behavior of the particle? -->

In [ ]:
plot_height = np.max(np.abs(qm.psi))
for n in range(100):
    print(n, end='\r')
    qm.evolve_schrodinger(20)
    fig, ax = qm.plot_complex_field(qm.psi, size=(400, 400)) # Plot the initial wavefunction
    qm.plot_field(qm.V_ext*plot_height, fig=fig, ax=ax, vlim=[0,plot_height])
    qm.plot_save(fig, n)
cf.tool_make_animation_gif(n)
print("Done")

#### Subproblem c)

- Change the width between the wells to half its original value. What happens to the tunneling probability now?